# Hoja de trabajo Redes Neuronales

Importamos todas las librerias necesarias

In [231]:
import numpy as np
from scipy import optimize as op
import pandas as pd
import random
import pickle
from functools import reduce
from utils import mnist_reader

# Leer el dataset

Leemos el set de prueba

In [232]:
X_test, y_test = mnist_reader.load_mnist('data/fashion', kind='t10k')

Leemos el set de entrenamiento

In [233]:
X_train, y_train = mnist_reader.load_mnist('data/fashion', kind='train')

# Manipulación de datos

Convertimos a int64 los datos para manipularlos mejor

In [234]:
X_test = np.array(X_test, dtype=np.int64)
X_train = np.array(X_train, dtype=np.int64)

Normalizamos los datos

In [235]:
X_test = X_test / 1000.0
X_train = X_train / 1000.0

Ajustamos Y de prueba y de entrenamiento

In [236]:
m_test, n_train = X_test.shape
m_train, _ = X_train.shape

In [237]:
y_test = y_test.reshape(m_test, 1)
y_train = y_train.reshape(m_train, 1)

# Definimos la estructura de la red neuronal

La estructura de esta red neuronal consta de;  una capa de entrada de 784 neuronas dadas por los píxeles de la imagen.
una capa escondida de 125 neuronas, y una capa de salida de 10 neuronas que son el total de prendas posibles.

In [238]:
nn = np.array( [ n_train, 125, 10 ] )

# Definimos funciones para la optminazación de los datos

Matriz theta formada por el modelo de red neuronal

In [239]:
theta_shapes  = np.hstack((
     nn[1:].reshape(len(nn) - 1, 1),
    (nn[:-1] + 1).reshape(len(nn) - 1, 1)
))

Creamos una matriz de pesos random 

In [240]:
flatten_list_of_arrays = lambda list_of_arrays: reduce(
    lambda acc, v: np.array([*acc.flatten(), *v.flatten()]),
    list_of_arrays
)

In [241]:
flat_thetas = flatten_list_of_arrays([
    np.random.rand(*theta_shape)
    for theta_shape in theta_shapes
])

Obtenemos la matriz del label que es el nombre de la prenda

In [242]:
Y = (y_train == np.array(range(10))).astype(int)

Función para "inflar" arrays

In [243]:
def inflate_matrixes(flat_thetas, shapes):
    layers = len(shapes) + 1
    sizes = [shape[0] * shape[1] for shape in shapes]
    steps = np.zeros(layers, dtype=int)

    for i in range(layers - 1):
        steps[i + 1] = steps[i] + sizes[i]

    return [
        flat_thetas[steps[i]: steps[i + 1]].reshape(*shapes[i])
        for i in range(layers - 1)
    ]

Función de costo

In [244]:
def cost_function(flat_thetas, shapes, X, Y):
    a = feed_forward(
        inflate_matrixes(flat_thetas, shapes),
        X
    )
    return -(Y * np.log(a[-1]) + (1 - Y) * np.log(1 - a[-1])).sum() / len(X)

Función Sigmoide

In [245]:
def sigmoid(z):
    a = [(1 / (1 + np.exp(-x))) for x in z]
    return np.asarray(a).reshape(z.shape)

Implementación del algoritmos Back Propagation

In [246]:
def back_propagation(flat_thetas, shapes, X, Y):
    m, layers = len(X), len(shapes) + 1
    thetas = inflate_matrixes(flat_thetas, shapes)
    
    a = feed_forward(thetas, X)

    deltas = [*range(layers - 1), a[-1] - Y]
    for i in range(layers - 2, 0, -1):
        deltas[i] = (deltas[i + 1] @ np.delete(thetas[i], 0, 1)) * (a[i] * (1 - a[i]))

    Deltas = []
    for i in range(layers - 1):
        Deltas.append(
            (deltas[i + 1].T
            @
            np.hstack((
                np.ones(len(a[i])).reshape(len(a[i]), 1),
                a[i]
            ))) / m
        )
    Deltas = np.asarray(Deltas)

    return flatten_list_of_arrays(
        Deltas
    )

Implementación del algoritmo Feed Forward

In [247]:
def feed_forward(thetas, X):
    MatrizA = [np.asarray(X)]

    for i in range(len(thetas)):
        MatrizA.append(
            sigmoid(
                np.matmul(
                    np.hstack((
                        np.ones(len(X)).reshape(len(X), 1),
                        MatrizA[i]
                    )), thetas[i].T
                )
            )            
        )
    return MatrizA

# Optimizamos los datos

In [248]:
optimized_thetas = op.minimize(
    fun = cost_function,
    x0 = flat_thetas,
    args = (theta_shapes, X_train, Y),
    method = 'L-BFGS-B',
    jac = back_propagation,
    options = {'disp': True, 'maxiter': 3000}
)

np.savetxt("data/optimizedThetas.txt", optimized_thetas.x)

# Rendimiento del modelo

Optenemos las thetas previamente optimizadas

In [249]:
optimized_thetas = np.loadtxt("data/optimizedThetas.txt")

Creamos la matriz de thetas

In [250]:
thetas = inflate_matrixes(
    optimized_thetas,
    theta_shapes
)

Implementamos feed forward

In [251]:
ff_res = feed_forward(
    thetas,
    X_test
)

# Resultado de los datos

In [252]:
predict_label = np.argmax(ff_res[-1], axis = 1)

In [253]:
accuracy = 0

for i in range(len(ff_res[-1])):
    if(predict_label[i] == y_test[i][0]):
        accuracy += 1
        
print("Total Accuracy: ",  round(100* accuracy/length) ,"%")

Total Accuracy:  85 %


# Conclusión

Se obtuvo un porcentaje de éxito del 85% lo cual es un porcentaje alto y podemos concluir que obtuvimos un buen modelo.